# Sentiment Analysis for Amazon Personal care applicances Customer review

Checking the GPU availability

In [0]:
!nvidia-smi

Wed Apr  1 19:37:08 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8     7W /  75W |      0MiB /  7611MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

Importing required libraries

In [0]:
from __future__ import absolute_import,division,print_function,unicode_literals

try:
  !pip uninstall tb-nightly tensorboardX tensorboard
  !pip install tf-nightly
except:
  pass
import tensorflow as tf
import os
import datetime
import tensorflow_datasets as tfds

%load_ext tensorboard


Uninstalling tb-nightly-2.3.0a20200401:
  Would remove:
    /usr/local/bin/tensorboard
    /usr/local/lib/python3.6/dist-packages/tb_nightly-2.3.0a20200401.dist-info/*
    /usr/local/lib/python3.6/dist-packages/tensorboard/*
Proceed (y/n)? y
  Successfully uninstalled tb-nightly-2.3.0a20200401
  Using cached https://files.pythonhosted.org/packages/40/38/102f35a6b0e63081b2180e8abb6fc1c656d9b3b96695425b2007e6727972/tb_nightly-2.3.0a20200401-py3-none-any.whl


In [0]:

import tensorflow as tf
import os
import datetime
import tensorflow_datasets as tfds

%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [0]:
import pkg_resources
for entry_point in pkg_resources.iter_entry_points('tensorboard_plugins'):
     print(entry_point)

wit = tensorboard_plugin_wit.wit_plugin_loader:WhatIfToolPluginLoader
projector = tensorboard.plugins.projector.projector_plugin:ProjectorPlugin


In [0]:
print(tf.__version__)

2.2.0-dev20200401


In [0]:
tf.config.experimental.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'),
 PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

Loading the data

In [0]:
dataset,info=tfds.load('amazon_us_reviews/Personal_Care_Appliances_v1_00',with_info=True)
train_dataset=dataset['train']

In [0]:
info

tfds.core.DatasetInfo(
    name='amazon_us_reviews',
    version=0.1.0,
    description='Amazon Customer Reviews (a.k.a. Product Reviews) is one of Amazons iconic products. In a period of over two decades since the first review in 1995, millions of Amazon customers have contributed over a hundred million reviews to express opinions and describe their experiences regarding products on the Amazon.com website. This makes Amazon Customer Reviews a rich source of information for academic researchers in the fields of Natural Language Processing (NLP), Information Retrieval (IR), and Machine Learning (ML), amongst others. Accordingly, we are releasing this data to further research in multiple disciplines related to understanding customer product experiences. Specifically, this dataset was constructed to represent a sample of customer evaluations and opinions, variation in the perception of a product across geographical regions, and promotional intent or bias in reviews.

Over 130+ million cus

In [0]:
print(train_dataset)

<DatasetV1Adapter shapes: {data: {customer_id: (), helpful_votes: (), marketplace: (), product_category: (), product_id: (), product_parent: (), product_title: (), review_body: (), review_date: (), review_headline: (), review_id: (), star_rating: (), total_votes: (), verified_purchase: (), vine: ()}}, types: {data: {customer_id: tf.string, helpful_votes: tf.int32, marketplace: tf.string, product_category: tf.string, product_id: tf.string, product_parent: tf.string, product_title: tf.string, review_body: tf.string, review_date: tf.string, review_headline: tf.string, review_id: tf.string, star_rating: tf.int32, total_votes: tf.int32, verified_purchase: tf.int64, vine: tf.int64}}>


In [0]:
len(list(train_dataset))

85981

Setting the buffer-size and batch size for training

In [0]:
BUFFER_SIZE=30000
BATCH_SIZE=64

In [0]:
train_dataset=train_dataset.shuffle(BUFFER_SIZE,reshuffle_each_iteration=False)

In [0]:
for reviews in train_dataset.take(2):
  print(reviews)

{'data': {'customer_id': <tf.Tensor: shape=(), dtype=string, numpy=b'52663666'>, 'helpful_votes': <tf.Tensor: shape=(), dtype=int32, numpy=6>, 'marketplace': <tf.Tensor: shape=(), dtype=string, numpy=b'US'>, 'product_category': <tf.Tensor: shape=(), dtype=string, numpy=b'Personal_Care_Appliances'>, 'product_id': <tf.Tensor: shape=(), dtype=string, numpy=b'B00012KDRM'>, 'product_parent': <tf.Tensor: shape=(), dtype=string, numpy=b'563396013'>, 'product_title': <tf.Tensor: shape=(), dtype=string, numpy=b'Vibralite 2 with Plain Band'>, 'review_body': <tf.Tensor: shape=(), dtype=string, numpy=b"My husband is a pilot who flies international.  He needed a silent alarm to wake him up when he took his turn napping on the plane to Europe.  After using this watch several times now for this purpose, he couldn't be happier.  The vibration is strong enough to wake him without waking anyone else.  And he likes that it is easy to turn off.  The design is simple enough that it is fine to wear with his

In [0]:
for reviews in train_dataset.take(2):
    review_text=reviews['data']
    print(review_text.get('review_body').numpy())
    print(tf.where(review_text.get('star_rating')>3,1,0).numpy())


b"My husband is a pilot who flies international.  He needed a silent alarm to wake him up when he took his turn napping on the plane to Europe.  After using this watch several times now for this purpose, he couldn't be happier.  The vibration is strong enough to wake him without waking anyone else.  And he likes that it is easy to turn off.  The design is simple enough that it is fine to wear with his uniform.  Since it arrived quickly after ordering, well-packaged and in perfect condition, we are both very pleased!"
1
b'The iWalk has free me from my crutches, and a Rolland around cart.  I use it around the house and at work.'
1


In [0]:
tokenizer=tfds.features.text.Tokenizer()
vocabulary_set=set()

for _,reviews in train_dataset.enumerate():
  review_text=reviews['data']
  review_tokens=tokenizer.tokenize(review_text.get('review_body').numpy())
  vocabulary_set.update(review_tokens)
  

In [0]:
 vocab_size=len(vocabulary_set)
 vocab_size

68782

# Creating dictionary for the words in training data set

In [0]:
encoder=tfds.features.text.TokenTextEncoder(vocabulary_set)

In [0]:
print(vocabulary_set)

{'advocate', 'ANIMALS', 'Theracane', 'Sayers', 'Backup', 'BATTERY', 'scrolling', 'exacltly', 'Aloe', 'Night', 'respirators', 'Michigan', 'finesse', 'Smartphones', 'defectives', 'Itchy', 'commence', 'MASOCHIST', 'Mondays', 'BLUEBIRD', 'Pile', '1531', 'infrequently', 'Athletes', 'frezze', 'Premiere', 'REFUSE', 'DISCONTINUE', 'COMMERCIAL', 'Droid', 'await', 'Welch', 'senged', 'amd', 'lupus', 'frantically', 'craning', 'boro', 'BRAND', 'Wearing', 'AFFORDABLE', 'Apparel', 'forked', 'Rules', 'aerea', 'RX330', 'foggy', 'prolonged', 'METHODS', 'hehe', 'EXCHANGE', '25lbs', 'infect', 'wheeler', 'WOrks', 'Spark', 'Herbst', 'Airhawk', '5YRS', 'dulling', 'splints', '89lb', 'asburd', 'purifying', 'Wait', 'Endolane', 'discovery', 'Contact', 'costo', 'B00006WLCC', 'INDICATED', 'leaved', 'Wein', 'menstrual', 'kgs', 'perky', 'Oximeter', 'Doctors', 'flickers', 'FLAT', 'quadrant', 'Balmex', 'handwork', 'Empi', 'shrivel', 'bacteria', 'doth', 'intention', 'PreciseHearing', 'iSmooth', 'workks', 'breaches', 'm

In [0]:
for reviews in train_dataset.take(2):
  review_text=reviews['data']
  print(review_text.get('review_body').numpy())
  encoded_example=encoder.encode(review_text.get('review_body').numpy())
  print(encoded_example)

b"My husband is a pilot who flies international.  He needed a silent alarm to wake him up when he took his turn napping on the plane to Europe.  After using this watch several times now for this purpose, he couldn't be happier.  The vibration is strong enough to wake him without waking anyone else.  And he likes that it is easy to turn off.  The design is simple enough that it is fine to wear with his uniform.  Since it arrived quickly after ordering, well-packaged and in perfect condition, we are both very pleased!"
[36612, 62120, 42946, 28309, 20308, 59531, 3666, 31640, 7669, 51704, 28309, 2427, 60089, 60364, 58944, 24692, 4049, 18338, 10546, 34367, 2469, 4897, 25631, 67407, 3511, 16336, 60364, 13885, 39767, 64964, 33499, 55308, 17834, 9134, 45301, 28837, 33499, 47837, 10546, 55189, 23190, 5208, 59437, 42842, 17726, 42946, 49148, 32870, 60364, 58944, 24692, 15238, 43815, 54499, 50959, 5333, 10546, 15569, 51469, 27307, 42946, 1457, 60364, 4897, 59650, 42842, 10172, 42946, 48812, 32870

In [0]:
for index in encoded_example:
  print('{} ----> {}'.format(index,encoder.decode([index])))

42842 ----> The
46460 ----> iWalk
53582 ----> has
5336 ----> free
26906 ----> me
64941 ----> from
12127 ----> my
41670 ----> crutches
4911 ----> and
28309 ----> a
32591 ----> Rolland
26839 ----> around
35101 ----> cart
31414 ----> I
42459 ----> use
27307 ----> it
26839 ----> around
3511 ----> the
7680 ----> house
4911 ----> and
43434 ----> at
33742 ----> work


# Function to encode words in the dataset and create its corresponding labels


In [0]:
def encode(text_tensor,label_tensor):
  encoded_text=encoder.encode(text_tensor.numpy())
  label=tf.where(label_tensor>3,1,0)
  return encoded_text,label

In [0]:
def encoded_map_fn(tensor):
  text=tensor['data'].get('review_body')
  label=tensor['data'].get('star_rating')
  encoded_text,label=tf.py_function(encode,inp=[text,label],Tout=(tf.int64,tf.int32))
  encoded_text.set_shape([None])
  label.set_shape([])
  return encoded_text,label

In [0]:
ar_encoded_data=train_dataset.map(encoded_map_fn)

Checking the training dataset inputs

In [0]:
for f0,f1 in ar_encoded_data.take(2):
  print(f0)
  print(f1)

tf.Tensor(
[36612 62120 42946 28309 20308 59531  3666 31640  7669 51704 28309  2427
 60089 60364 58944 24692  4049 18338 10546 34367  2469  4897 25631 67407
  3511 16336 60364 13885 39767 64964 33499 55308 17834  9134 45301 28837
 33499 47837 10546 55189 23190  5208 59437 42842 17726 42946 49148 32870
 60364 58944 24692 15238 43815 54499 50959  5333 10546 15569 51469 27307
 42946  1457 60364  4897 59650 42842 10172 42946 48812 32870 51469 27307
 42946 11239 60364  6049 34088  2469 54652 41049 27307 28072 26265 40817
 10688  5541 17826  4911 68327 50658 30443  7956 45194  8175 27635 35253], shape=(96,), dtype=int64)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(
[42842 46460 53582  5336 26906 64941 12127 41670  4911 28309 32591 26839
 35101 31414 42459 27307 26839  3511  7680  4911 43434 33742], shape=(22,), dtype=int64)
tf.Tensor(1, shape=(), dtype=int32)


In [0]:
Take_size=10000
train_data=ar_encoded_data.skip(Take_size).shuffle(BUFFER_SIZE)
train_data=train_data.padded_batch(BATCH_SIZE)

test_data=ar_encoded_data.take(Take_size)
test_data=test_data.padded_batch(BATCH_SIZE)

In [0]:
vocab_size+=1

In [0]:
sample_text,sample_labels=next(iter(test_data))
sample_text[0],sample_labels[0]

(<tf.Tensor: shape=(407,), dtype=int64, numpy=
 array([36612, 62120, 42946, 28309, 20308, 59531,  3666, 31640,  7669,
        51704, 28309,  2427, 60089, 60364, 58944, 24692,  4049, 18338,
        10546, 34367,  2469,  4897, 25631, 67407,  3511, 16336, 60364,
        13885, 39767, 64964, 33499, 55308, 17834,  9134, 45301, 28837,
        33499, 47837, 10546, 55189, 23190,  5208, 59437, 42842, 17726,
        42946, 49148, 32870, 60364, 58944, 24692, 15238, 43815, 54499,
        50959,  5333, 10546, 15569, 51469, 27307, 42946,  1457, 60364,
         4897, 59650, 42842, 10172, 42946, 48812, 32870, 51469, 27307,
        42946, 11239, 60364,  6049, 34088,  2469, 54652, 41049, 27307,
        28072, 26265, 40817, 10688,  5541, 17826,  4911, 68327, 50658,
        30443,  7956, 45194,  8175, 27635, 35253,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,  

In [0]:
for f0,f1 in test_data.take(10):
  print(tf.unique_with_counts(f1)[2].numpy())

[49 15]
[51 13]
[23 41]
[47 17]
[43 21]
[43 21]
[19 45]
[44 20]
[46 18]
[42 22]


# Creating a bi-directional lstm model

In [0]:
model=tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(vocab_size,128))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128,return_sequences=True)))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)))
for units in [64,64]:
  model.add(tf.keras.layers.Dense(units,activation='relu'))
model.add(tf.keras.layers.Dense(1))



In [0]:
!rm -r /tmp/logs

rm: cannot remove '/tmp/logs': No such file or directory


In [0]:
#logdir=os.path.join("/tmp/logs",datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

In [0]:
#tensorboard_callback=tf.keras.callbacks.TensorBoard(logdir,histogram_freq=1)

# Creating Check-points to save the best model and giving inputs to the lstm model for compiling it

In [0]:
checkpointer=tf.keras.callbacks.ModelCheckpoint(filepath='/tmp/sentiment_analysis.hdf5',verbose=1,save_best_only=True)

In [0]:
model.compile(optimizer='adam',loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),metrics=['accuracy'])

In [0]:
!nvidia-smi

Wed Apr  1 19:38:25 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P0    23W /  75W |    225MiB /  7611MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
model.fit(train_data,epochs=4,validation_data=test_data,callbacks=[checkpointer])

Epoch 1/4
1188/1188 [==============================] - ETA: 0s - loss: 0.3810 - accuracy: 0.8247
Epoch 00001: val_loss improved from inf to 0.28760, saving model to /tmp/sentiment_analysis.hdf5
1188/1188 [==============================] - 305s 257ms/step - loss: 0.3810 - accuracy: 0.8247 - val_loss: 0.2876 - val_accuracy: 0.8816
Epoch 2/4
1188/1188 [==============================] - ETA: 0s - loss: 0.2284 - accuracy: 0.9059
Epoch 00002: val_loss improved from 0.28760 to 0.26051, saving model to /tmp/sentiment_analysis.hdf5
1188/1188 [==============================] - 308s 259ms/step - loss: 0.2284 - accuracy: 0.9059 - val_loss: 0.2605 - val_accuracy: 0.8902
Epoch 3/4
1188/1188 [==============================] - ETA: 0s - loss: 0.1661 - accuracy: 0.9358
Epoch 00003: val_loss did not improve from 0.26051
1188/1188 [==============================] - 305s 257ms/step - loss: 0.1661 - accuracy: 0.9358 - val_loss: 0.3042 - val_accuracy: 0.8892
Epoch 4/4
1188/1188 [============================

In [0]:
model.save('/tmp/final_sentiment_analysis.hdf5')

In [0]:
!ls -alrt /tmp/*.hdf5

-rw-r--r-- 1 root root 111020544 Apr  1 19:49 /tmp/sentiment_analysis.hdf5
-rw-r--r-- 1 root root 111020544 Apr  1 20:00 /tmp/final_sentiment_analysis.hdf5


In [0]:
eval_loss,eval_acc=model.evaluate(test_data)

157/157 [==============================] - 16s 104ms/step - loss: 0.3475 - accuracy: 0.8888


In [0]:
print('\n Eval loss:{:.3f},Eval accuracy:{:.3f}'.format(eval_loss,eval_acc))


 Eval loss:0.347,Eval accuracy:0.889


In [0]:
model.layers

In [0]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 128)         8804224   
_________________________________________________________________
bidirectional (Bidirectional (None, None, 256)         263168    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 128)               164352    
_________________________________________________________________
dense (Dense)                (None, 64)                8256      
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total params: 9,244,225
Trainable params: 9,244,225
Non-trainable params: 0
______________________________________________

In [0]:
model.get_layer('embedding').output

<tf.Tensor 'embedding/Identity:0' shape=(None, None, 128) dtype=float32>

In [0]:
!nvidia-smi

Wed Apr  1 20:03:01 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P0    24W /  75W |   7359MiB /  7611MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

# Checking the model output and compare against actual output

In [0]:
for f0,f1 in test_data.take(1):
  print(f1)
  print(model.predict(f0))

tf.Tensor(
[1 1 1 1 1 0 0 1 1 1 0 0 1 0 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1
 1 1 0 1 1 1 1 0 1 0 1 0 1 1 1 0 1 1 1 1 1 0 1 1 0 1 1], shape=(64,), dtype=int32)
[[ 6.273405  ]
 [ 3.9164836 ]
 [ 5.19577   ]
 [ 6.308716  ]
 [ 1.4491527 ]
 [-3.6236963 ]
 [-3.7970917 ]
 [ 5.6132183 ]
 [ 5.725909  ]
 [ 5.284954  ]
 [ 0.5896925 ]
 [-3.90121   ]
 [ 6.218645  ]
 [-3.8954422 ]
 [ 6.431924  ]
 [ 3.7601194 ]
 [ 5.378118  ]
 [-4.9772778 ]
 [ 6.0493445 ]
 [ 1.9640954 ]
 [ 5.482977  ]
 [ 1.6559052 ]
 [ 4.1508827 ]
 [ 5.852295  ]
 [ 6.7949095 ]
 [-5.0494976 ]
 [ 3.5322025 ]
 [ 5.217817  ]
 [ 3.9494717 ]
 [-0.6585846 ]
 [ 6.1413627 ]
 [ 5.3195868 ]
 [ 1.9881806 ]
 [ 4.9940825 ]
 [ 5.1582155 ]
 [ 6.8692265 ]
 [ 6.28662   ]
 [ 6.2286844 ]
 [ 6.500596  ]
 [-0.12740351]
 [ 1.4020796 ]
 [ 6.405269  ]
 [ 6.7149677 ]
 [ 6.1333933 ]
 [-3.9097857 ]
 [ 5.8973103 ]
 [-4.3644457 ]
 [ 3.03506   ]
 [-4.4823093 ]
 [ 6.2758336 ]
 [ 3.6190002 ]
 [ 6.345031  ]
 [-2.1579704 ]
 [ 6.0636635 ]
 [ 6.190802  ]
 [ 5.3

# Procedure for testing the model

In [0]:
tf.keras.backend.clear_session()
sa_load=tf.keras.models.load_model('/tmp/sentiment_analysis.hdf5',compile=False)

Function to pad the test data such that all inputs have same length when fed to the lstm network

In [0]:
def pad_to_size(vec,size):
  zeros=[0]*(size-len(vec))
  vec.extend(zeros)
  return vec

Function to predict the sentiment of the given customer review

In [0]:
def predict_fn(pred_text):
  encoded_pred_text=encoder.encode(pred_text)
  print(encoded_pred_text)
  encoded_pred_text=pad_to_size(encoded_pred_text,32)
  print(encoded_pred_text)
  encoded_pred_text=tf.cast(encoded_pred_text,tf.float64)
  predictions=sa_load.predict(tf.expand_dims(encoded_pred_text,0))
  return predictions


In [0]:
pred_text=('The iWalk has not free me from my crutches.  It is pain to use around the house and at work.')
predictions=predict_fn(pred_text)
print(predictions)

[42842, 46460, 53582, 399, 5336, 26906, 64941, 12127, 41670, 44395, 42946, 9634, 60364, 42459, 26839, 3511, 7680, 4911, 43434, 33742]
[42842, 46460, 53582, 399, 5336, 26906, 64941, 12127, 41670, 44395, 42946, 9634, 60364, 42459, 26839, 3511, 7680, 4911, 43434, 33742, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[[0.7523117]]


In [0]:
pred_text=('The iphone is very good for personal use.  I use it around the house and at work.')
predictions=predict_fn(pred_text)
print(predictions)

[42842, 19653, 42946, 27635, 42169, 28837, 32162, 42459, 31414, 42459, 27307, 26839, 3511, 7680, 4911, 43434, 33742]
[42842, 19653, 42946, 27635, 42169, 28837, 32162, 42459, 31414, 42459, 27307, 26839, 3511, 7680, 4911, 43434, 33742, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[[1.5906576]]
